In [28]:
import numpy as np
from sklearn.datasets import load_iris

# 1. Funciones auxiliares
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))

# 2. Inicialización de pesos
np.random.seed(42)
input_neurons = 4  # Características de entrada
hidden_neurons = 5  # Neuronas en capa oculta
output_neurons = 1  # Clasificación binaria (1 salida)

W1 = np.random.rand(input_neurons, hidden_neurons) - 0.5
b1 = np.zeros((1, hidden_neurons))
W2 = np.random.rand(hidden_neurons, output_neurons) - 0.5
b2 = np.zeros((1, output_neurons))

# 3. Propagación hacia adelante
def forward_propagation(X):
    Z1 = np.dot(X, W1) + b1
    A1 = sigmoid(Z1)
    Z2 = np.dot(A1, W2) + b2
    A2 = sigmoid(Z2)
    return Z1, A1, Z2, A2

# 4. Retropropagación y actualización
def backpropagation(X, y, Z1, A1, Z2, A2):
    global W1, b1, W2, b2
    m = X.shape[0]  # Número de ejemplos

    # Gradientes para la capa de salida
    dZ2 = A2 - y
    dW2 = np.dot(A1.T, dZ2) / m
    db2 = np.sum(dZ2, axis=0, keepdims=True) / m

    # Gradientes para la capa oculta
    dA1 = np.dot(dZ2, W2.T)
    dZ1 = dA1 * sigmoid_derivative(Z1)
    dW1 = np.dot(X.T, dZ1) / m
    db1 = np.sum(dZ1, axis=0, keepdims=True) / m

    # Actualizar pesos
    W1 -= 0.4 * dW1
    b1 -= 0.4 * db1
    W2 -= 0.4 * dW2
    b2 -= 0.4 * db2

# 5. Entrenamiento
def train(X, y, epochs):
    for epoch in range(epochs):
        Z1, A1, Z2, A2 = forward_propagation(X)
        backpropagation(X, y, Z1, A1, Z2, A2)
        if epoch % 10 == 0:
            loss = np.mean((A2 - y) ** 2)  # Error cuadrático medio
            print(f"Epoch {epoch}, Loss: {loss}")

# 6. Cargar y preparar dataset Iris
data = load_iris()
X = data.data
y = (data.target != 0).astype(int).reshape(-1, 1)  # Clasificación binaria: setosa vs otras

# Normalizar manualmente las características (0 a 1)
X_min = np.min(X, axis=0)
X_max = np.max(X, axis=0)
X = (X - X_min) / (X_max - X_min)

# Dividir manualmente en conjunto de entrenamiento y prueba
def train_test_split_manual(X, y, test_size=0.2, random_seed=42):
    np.random.seed(random_seed)
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)
    split_idx = int(X.shape[0] * (1 - test_size))
    train_idx, test_idx = indices[:split_idx], indices[split_idx:]
    return X[train_idx], X[test_idx], y[train_idx], y[test_idx]

X_train, X_test, y_train, y_test = train_test_split_manual(X, y, test_size=0.2)

# 7. Entrenar la red
train(X_train, y_train, epochs=112)

# 8. Evaluación
def evaluate(X, y):
    _, _, _, A2 = forward_propagation(X)
    predictions = (A2 > 0.5).astype(int)
    accuracy = np.mean(predictions == y)
    return accuracy

accuracy = evaluate(X_test, y_test)
print(f"Accuracy on test set: {accuracy * 100:.2f}%")


Epoch 0, Loss: 0.2882639617270294
Epoch 10, Loss: 0.23274089585755273
Epoch 20, Loss: 0.22872712809928733
Epoch 30, Loss: 0.22501112322333824
Epoch 40, Loss: 0.2205797177835229
Epoch 50, Loss: 0.21485418043054447
Epoch 60, Loss: 0.2072395620206788
Epoch 70, Loss: 0.19716336512232796
Epoch 80, Loss: 0.18417871471534905
Epoch 90, Loss: 0.16812154941907537
Epoch 100, Loss: 0.1492917765384915
Epoch 110, Loss: 0.12857671328471207
Accuracy on test set: 100.00%


In [31]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import StratifiedKFold

# 1. Funciones auxiliares (sin cambios)
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))

# 2. Inicialización de pesos (sin cambios)
def initialize_weights(input_neurons, hidden_neurons, output_neurons):
    W1 = np.random.rand(input_neurons, hidden_neurons) - 0.5
    b1 = np.zeros((1, hidden_neurons))
    W2 = np.random.rand(hidden_neurons, output_neurons) - 0.5
    b2 = np.zeros((1, output_neurons))
    return W1, b1, W2, b2

# 3. Propagación hacia adelante (sin cambios)
def forward_propagation(X, W1, b1, W2, b2):
    Z1 = np.dot(X, W1) + b1
    A1 = sigmoid(Z1)
    Z2 = np.dot(A1, W2) + b2
    A2 = sigmoid(Z2)
    return Z1, A1, Z2, A2

# 4. Retropropagación y actualización (sin cambios)
def backpropagation(X, y, Z1, A1, Z2, A2, W1, b1, W2, b2, learning_rate=0.4):
    m = X.shape[0]  # Número de ejemplos

    # Gradientes para la capa de salida
    dZ2 = A2 - y
    dW2 = np.dot(A1.T, dZ2) / m
    db2 = np.sum(dZ2, axis=0, keepdims=True) / m

    # Gradientes para la capa oculta
    dA1 = np.dot(dZ2, W2.T)
    dZ1 = dA1 * sigmoid_derivative(Z1)
    dW1 = np.dot(X.T, dZ1) / m
    db1 = np.sum(dZ1, axis=0, keepdims=True) / m

    # Actualizar pesos
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2

    return W1, b1, W2, b2

# 5. Entrenamiento del modelo (sin cambios)
def train(X, y, epochs, W1, b1, W2, b2):
    for epoch in range(epochs):
        Z1, A1, Z2, A2 = forward_propagation(X, W1, b1, W2, b2)
        W1, b1, W2, b2 = backpropagation(X, y, Z1, A1, Z2, A2, W1, b1, W2, b2)
    return W1, b1, W2, b2

# 6. Evaluación del modelo (sin cambios)
def evaluate(X, y, W1, b1, W2, b2):
    _, _, _, A2 = forward_propagation(X, W1, b1, W2, b2)
    predictions = (A2 > 0.5).astype(int)
    accuracy = np.mean(predictions == y)
    return accuracy

# 7. Validación cruzada k-fold con estratificación
def cross_validation(X, y, k=5, epochs=100):
    skf = StratifiedKFold(n_splits=k)
    accuracies = []

    for i, (train_index, val_index) in enumerate(skf.split(X, y)):
        # Dividir el dataset en entrenamiento y validación de forma estratificada
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]

        # Inicializar pesos
        W1, b1, W2, b2 = initialize_weights(input_neurons=4, hidden_neurons=5, output_neurons=1)

        # Entrenar el modelo
        W1, b1, W2, b2 = train(X_train, y_train, epochs, W1, b1, W2, b2)

        # Evaluar el modelo en el conjunto de validación
        accuracy = evaluate(X_val, y_val, W1, b1, W2, b2)
        accuracies.append(accuracy)
        print(f"Fold {i + 1}, Accuracy: {accuracy * 100:.2f}%")

    # Promedio de las precisiones
    avg_accuracy = np.mean(accuracies)
    print(f"Average Accuracy: {avg_accuracy * 100:.2f}%")
    return avg_accuracy

# 8. Cargar y preparar el dataset Iris
data = load_iris()
X = data.data
y = (data.target != 0).astype(int).reshape(-1, 1)  # Clasificación binaria: setosa vs otras

# Normalizar manualmente las características (0 a 1)
X_min = np.min(X, axis=0)
X_max = np.max(X, axis=0)
X = (X - X_min) / (X_max - X_min)

# 9. Realizar la validación cruzada
cross_validation(X, y, k=5, epochs=100)


Fold 1, Accuracy: 100.00%
Fold 2, Accuracy: 80.00%
Fold 3, Accuracy: 86.67%
Fold 4, Accuracy: 80.00%
Fold 5, Accuracy: 66.67%
Average Accuracy: 82.67%


0.8266666666666668